In [2]:
import pandas as pd
df = pd.read_csv(
    "data/output/2024-02.csv", 
    usecols=["dataset_id", "dno_alias", "secondary_substation_id", "secondary_substation_name", "lv_feeder_id", "lv_feeder_name", "aggregated_device_count_active", "total_consumption_active_import", "data_collection_log_timestamp", "substation_latitude", "substation_longitude"],
    dtype={"dataset_id": str, "dno_alias": str, "secondary_substation_id": str, "secondary_substation_name": str, "lv_feeder_id": str, "lv_feeder_name": str, "aggregated_device_count_active": int, "total_consumption_active_import": int, "data_collection_log_timestamp": str, "substation_latitude": float, "substation_longitude": float}
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97416304 entries, 0 to 97416303
Data columns (total 11 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   dataset_id                       object 
 1   dno_alias                        object 
 2   secondary_substation_id          object 
 3   secondary_substation_name        object 
 4   lv_feeder_id                     object 
 5   lv_feeder_name                   object 
 6   aggregated_device_count_active   int64  
 7   total_consumption_active_import  int64  
 8   data_collection_log_timestamp    object 
 9   substation_latitude              float64
 10  substation_longitude             float64
dtypes: float64(2), int64(2), object(7)
memory usage: 8.0+ GB


In [3]:
import datetime
import geopandas as gpd
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.substation_longitude, df.substation_latitude, crs="EPSG:4326"))
del df
gdf["data_collection_log_timestamp"] = gdf.data_collection_log_timestamp.map(lambda x: datetime.datetime.fromisoformat(x))

In [4]:
gdf["secondary_substation_unique_id"] = gdf["secondary_substation_id"] + "-" + gdf["secondary_substation_name"]
gdf["secondary_substation_unique_id"] = gdf["secondary_substation_unique_id"].str.rstrip("-")
gdf["lv_feeder_unique_id"] = gdf["lv_feeder_id"] + "-" + gdf["lv_feeder_name"]
gdf["lv_feeder_unique_id"] = gdf["lv_feeder_unique_id"].str.rstrip("-")
gdf.drop(columns=["secondary_substation_id", "secondary_substation_name", "lv_feeder_id", "lv_feeder_name", "substation_longitude", "substation_latitude"], inplace=True)

In [5]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 97416304 entries, 0 to 97416303
Data columns (total 8 columns):
 #   Column                           Dtype              
---  ------                           -----              
 0   dataset_id                       object             
 1   dno_alias                        object             
 2   aggregated_device_count_active   int64              
 3   total_consumption_active_import  int64              
 4   data_collection_log_timestamp    datetime64[ns, UTC]
 5   geometry                         geometry           
 6   secondary_substation_unique_id   object             
 7   lv_feeder_unique_id              object             
dtypes: datetime64[ns, UTC](1), geometry(1), int64(2), object(4)
memory usage: 5.8+ GB


In [6]:
gdf.sort_values(by=["data_collection_log_timestamp", "dno_alias", "secondary_substation_unique_id", "lv_feeder_unique_id"], inplace=True)

In [7]:
import pyarrow.parquet as pq
sorting_columns = [pq.SortingColumn(4), pq.SortingColumn(1), pq.SortingColumn(6), pq.SortingColumn(7)]
gdf.to_parquet("output/data/smart-meter.parquet", compression="zstd", compression_level=22, geometry_encoding="geoarrow", write_covering_bbox=True, index=False, schema_version="1.1.0", sorting_columns=sorting_columns, coerce_timestamps="ms", allow_truncated_timestamps=True, store_decimal_as_integer=True)